In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [5]:
test_trans = pd.read_csv('../data/transactions_test.csv', parse_dates=["TX_TS"])
train_trans =  pd.read_csv('../data/transactions_train.csv', parse_dates=["TX_TS"])
merchants_full = pd.read_csv('../data/merchants.csv')

terminals_full = pd.read_csv('../data/terminals.csv')

customers_full = pd.read_csv('../data/customers.csv')

C:\Users\joshu\AppData\Local\Temp\ipykernel_19328\1163954335.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  train_trans =  pd.read_csv('../data/transactions_train.csv', parse_dates=["TX_TS"])


In [4]:
test_trans['is_test'] = 1
train_trans['is_test'] = 0

In [5]:
all_trans = pd.concat([train_trans,test_trans]).drop(columns=['ID_JOIN'])

In [6]:
all_trans.columns

Index(['TX_ID', 'TX_TS', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'TX_FRAUD',
       'TRANSACTION_GOODS_AND_SERVICES_AMOUNT', 'TRANSACTION_CASHBACK_AMOUNT',
       'CARD_EXPIRY_DATE', 'CARD_DATA', 'CARD_BRAND', 'TRANSACTION_TYPE',
       'TRANSACTION_STATUS', 'FAILURE_CODE', 'FAILURE_REASON',
       'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE', 'MERCHANT_ID',
       'IS_RECURRING_TRANSACTION', 'ACQUIRER_ID', 'CARDHOLDER_AUTH_METHOD',
       'is_test'],
      dtype='object')

In [7]:
all_trans.sort_values('TX_TS')

,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,CARD_EXPIRY_DATE,CARD_DATA,...,TRANSACTION_STATUS,FAILURE_CODE,FAILURE_REASON,TRANSACTION_CURRENCY,CARD_COUNTRY_CODE,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,is_test
829437,a0d7c943b544215745e13ebce08a4b8276168767,2021-01-01 00:00:08+00:00,2108277906068006,27974316,78.98,0.0,78.98,0.0,04/21,4485********975,...,Settled,NaN,NaN,USD,FR,8cfe93a4-97ea-4247-8f15-50430fed3d88,True,ACQ1,Offline enciphered PIN,0
942865,c06aa650651e1402c16ddf309d61516de22d0331,2021-01-01 00:00:14+00:00,8662651220152351,46261637,46.42,0.0,46.42,0.0,04/21,4485********444,...,Settled,NaN,NaN,GBP,GB,c16a44e8-2e25-439f-ae29-e5e115c8c4e6,False,ACQ4,Signature,0
390644,09a966ffc5878eb652d3f36496cdb342d7576dd6,2021-01-01 00:00:15+00:00,9508987894574818,21664478,38.84,0.0,38.84,0.0,01/23,4916********672,...,Settled,NaN,NaN,EUR,DK,dbec5dd4-80d5-46df-b72c-0d9f15710e44,False,ACQ4,Signature,0
160094,5689648bed268990eb90c11fb9a7902f2d1b2cf2,2021-01-01 00:01:08+00:00,3040941832365513,60286185,57.74,0.0,57.74,0.0,07/22,4024********300,...,Settled,NaN,NaN,CHF,US,cf2b9bd5-7abd-42cf-bae6-eb82c14df28f,False,ACQ2,Signature,0
933901,541d5014993dafacfc25e54defd9c42a1cb580ce,2021-01-01 00:02:00+00:00,3014216432000074,56911243,99.13,0.0,99.13,0.0,03/22,4929********121,...,Settled,NaN,NaN,EUR,FR,fa2baef3-c782-42ce-9260-edb8778d305a,False,ACQ3,Offline enciphered PIN and signature,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29505,a2768a7f5787f21f973d445a42fb6354fc4444de,2021-12-07 23:59:49+00:00,8405719292105341,97869051,39.23,NaN,39.23,0.0,01/24,4916********261,...,Authorized,NaN,NaN,USD,US,43e0a0ba-bfae-4c63-83c4-59096298bc18,False,ACQ2,Offline plaintext PIN,1
46008,da7b5cfaeac2c75a4acd360f3acd9a360996d1ef,2021-12-07 23:59:50+00:00,238210135543937,78810971,94.33,NaN,94.33,0.0,02/23,5344********103,...,Settled,NaN,NaN,USD,FR,667b9826-15c0-4028-a6e5-6442a1f0cb6e,False,ACQ2,Online PIN,1
59242,8460cb3cdb17c8b6edfc0a463b7d3f2941934494,2021-12-07 23:59:57+00:00,2709822668400952,7490914,18.33,NaN,18.33,0.0,03/23,5337********388,...,Settled,NaN,NaN,USD,US,e61b06ad-6a45-4d95-b63f-2adfe6307992,False,ACQ2,Offline plaintext PIN,1
617787,8f265a9811bc630b476300ed78a61ae1e33ffc7d,2022-01-01 00:00:31+00:00,4724687086476235,64439061,73.44,1.0,73.44,0.0,10/22,4485********763,...,Settled,NaN,NaN,CHF,US,c7b9d4cc-51d0-43be-9645-7981ccc82f74,False,ACQ3,No CVM performed,0


## Data Cleaning

In [ ]:
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == 'True',['IS_RECURRING_TRANSACTION']] = 1
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == True,['IS_RECURRING_TRANSACTION']] = 1
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == 'False',['IS_RECURRING_TRANSACTION']] = 0
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == 'Fals',['IS_RECURRING_TRANSACTION']] = 0
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == False,['IS_RECURRING_TRANSACTION']] = 0

all_trans['IS_RECURRING_TRANSACTION'] = all_trans['IS_RECURRING_TRANSACTION'].apply(pd.to_numeric)

In [ ]:
all_trans.dropna(subset=['ACQUIRER_ID','CARDHOLDER_AUTH_METHOD'], inplace=True)

In [ ]:
useless_columns = ['CARD_EXPIRY_DATE','TX_ID','CARD_DATA']
all_trans.drop(columns=useless_columns,axis=1,inplace=True)

## Feature Engineering

In [78]:
bins = [0, 6, 12, 18, 24]
labels = [0, 6, 12, 18,24]
all_trans['Time_of_day'] = pd.cut(all_trans.TX_TS.dt.hour, bins, labels=labels, right=False)

bins = np.arange(12)
all_trans['Month'] = all_trans['TX_TS'].apply(lambda x: x.month)

all_trans['Day_of_week'] = all_trans['TX_TS'].apply(lambda x: x.weekday())


AttributeError: 'DataFrame' object has no attribute 'TX_TS'

We generally model customer behaviour through the RFM model
- Recency
- Frequency
- Monetary

In [12]:
def customer_behaviour(customer_transactions):
    
    customer_transactions=customer_transactions.sort_values('TX_TS')
    
    customer_transactions.index=customer_transactions.TX_TS
    
    for window_size in [1,7,30]: #By previous day, week and month
        
        WINDOW_SUM=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        WINDOW_COUNT=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        AVG_AMOUNT_TX_WINDOW=WINDOW_SUM/WINDOW_COUNT
    
        customer_transactions['C_ID_NB_TX_'+str(window_size)]=list(WINDOW_COUNT)
        customer_transactions['C_ID_AVG_AMOUNT_'+str(window_size)]=list(AVG_AMOUNT_TX_WINDOW)
    
    customer_transactions.index=customer_transactions.TX_ID
        
    return customer_transactions

In [13]:
# For each customer we get their previous behaviour
all_trans=all_trans.groupby('CUSTOMER_ID').apply(lambda x: customer_behaviour(x))
all_trans=all_trans.sort_values('TX_TS').reset_index(drop=True)

In [14]:
def get_terminal_risk(terminal_transactions):
    
    terminal_transactions=terminal_transactions.sort_values('TX_TS')
    
    terminal_transactions.index=terminal_transactions.TX_TS
    
    for window_size in [1,7,30]:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(7+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(7+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-(terminal_transactions['TX_FRAUD'].rolling(str(7)+'d').sum())
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-(terminal_transactions['TX_FRAUD'].rolling(str(7)+'d').count())
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions['TERMINAL_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions['TERMINAL_ID_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TX_ID
    
    terminal_transactions.fillna(0,inplace=True) #If we don't know risk factor set to 0
    
    return terminal_transactions

In [15]:
all_trans=all_trans.groupby('TERMINAL_ID').apply(lambda x: get_terminal_risk(x))
all_trans=all_trans.sort_values('TX_TS').reset_index(drop=True)

In [17]:
one_hot_encoded_col = ['CARD_BRAND','TRANSACTION_TYPE','TRANSACTION_STATUS','FAILURE_CODE','FAILURE_REASON','TRANSACTION_CURRENCY','CARD_COUNTRY_CODE','ACQUIRER_ID','CARDHOLDER_AUTH_METHOD']
all_trans = pd.get_dummies(all_trans, columns = one_hot_encoded_col)

In [99]:
test_features = (set(all_trans.columns)-set('TX_FRAUD'))
train_features = (set(all_trans.columns)-set('TX_ID'))

In [74]:
test_df = all_trans.loc[all_trans['is_test'] == 1][test_features]
train_df = all_trans.loc[all_trans['is_test'] == 0][train_features]

C:\Users\joshu\AppData\Local\Temp\ipykernel_18864\3799857911.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_df = all_trans.loc[all_trans['is_test'] == 1][test_features]
C:\Users\joshu\AppData\Local\Temp\ipykernel_18864\3799857911.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_df = all_trans.loc[all_trans['is_test'] == 0][train_features]


In [65]:
location = 'feature_set_6'
test_df.to_csv(F'../data/{location}/test_features.csv',index=False)

In [66]:
train_df.to_csv(F'../data/{location}/train_features.csv',index=False)

y = train_df['TX_FRAUD']
X = train_df.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

X_train_full.to_csv(F'../data/{location}/X_train_full.csv',index=False)
X_valid_full.to_csv(F'../data/{location}/X_valid_full.csv',index=False)
y_train.to_csv(F'../data/{location}/y_train.csv',index=False)
y_valid.to_csv(F'../data/{location}/y_valid.csv',index=False)

In [84]:
scaled_cols = all_trans.select_dtypes(include='float64').columns

In [83]:
scaler = MinMaxScaler()
scaler.fit(all_trans[ scaled_cols.columns])

MinMaxScaler()

In [92]:
non_scaled_cols = list(set(all_trans.columns)- set(scaled_cols))

In [89]:
all_trans_normalised = pd.DataFrame(scaler.transform(all_trans[list(scaled_cols)]), index=all_trans[list(scaled_cols)].index, columns=list(scaled_cols))

In [94]:
all_trans_normalised['TX_ID'] = all_trans['TX_ID']

In [95]:
all_trans_normalised = pd.merge(all_trans_normalised,all_trans[non_scaled_cols],on='TX_ID')

In [100]:
test_df = all_trans_normalised.loc[all_trans['is_test'] == 1][test_features]
train_df = all_trans_normalised.loc[all_trans['is_test'] == 0][train_features]

C:\Users\joshu\AppData\Local\Temp\ipykernel_18864\1029650358.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_df = all_trans_normalised.loc[all_trans['is_test'] == 1][test_features]
C:\Users\joshu\AppData\Local\Temp\ipykernel_18864\1029650358.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_df = all_trans_normalised.loc[all_trans['is_test'] == 0][train_features]


In [102]:
location = 'feature_set_6_normalised'
test_df.to_csv(F'../data/{location}/test_features.csv',index=False)

train_df.to_csv(F'../data/{location}/train_features.csv',index=False)

y = train_df['TX_FRAUD']
X = train_df.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

X_train_full.to_csv(F'../data/{location}/X_train_full.csv',index=False)
X_valid_full.to_csv(F'../data/{location}/X_valid_full.csv',index=False)
y_train.to_csv(F'../data/{location}/y_train.csv',index=False)
y_valid.to_csv(F'../data/{location}/y_valid.csv',index=False)

In [104]:
og_test = pd.read_csv('../data/transactions_test.csv', parse_dates=["TX_TS"])

In [105]:
set(og_test['TX_ID'])- set(test_df['TX_ID'])

set()